In [1]:
#Connecting to google drive
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/RS_Challenge/RSLib
%ls

/content/gdrive/My Drive/RS_Challenge/RSLib
RecSys_Course_AT_PoliMi-master/


In [3]:
!ls */*.py

RecSys_Course_AT_PoliMi-master/cython_example_FunkSVD.py
RecSys_Course_AT_PoliMi-master/cython_example_SLIM_MSE.py
RecSys_Course_AT_PoliMi-master/run_all_algorithms.py
RecSys_Course_AT_PoliMi-master/run_compile_all_cython.py
RecSys_Course_AT_PoliMi-master/run_hyperparameter_search.py
RecSys_Course_AT_PoliMi-master/run_test_recommenders.py


In [4]:
#Importing course's repositories
import sys
sys.path.append('/content/gdrive/My Drive/RS_Challenge/RSLib/RecSys_Course_AT_PoliMi-master')

In [5]:
#To use skopt
! pip install --quiet lightgbm==2.2.3 scikit-optimize==0.9.0 neptune-client==0.4.132 neptune-contrib[monitoring]==0.25.0

     |████████████████████████████████| 100 kB 8.1 MB/s 
     |████████████████████████████████| 110 kB 55.9 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 829 kB 34.0 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 180 kB 53.6 MB/s 
     |████████████████████████████████| 106 kB 72.3 MB/s 
     |████████████████████████████████| 175 kB 70.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 99 kB 10.4 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 130 kB 57.7 MB/s 
     |████████████████████████████████| 564 kB 47.0 MB/s 


In [6]:
#Importing libraries
import pandas as pd 
import scipy.sparse as sps
import numpy as np
import time
import matplotlib.pyplot as pyplot
import os
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

print("DEBUG")

DEBUG


In [7]:
%cd ..
%ls

/content/gdrive/My Drive/RS_Challenge
 RSDs/                     submission_MFIALS_full_URM.csv
 RSLib/                    submission_Round_Robin_2SLIM1MF_Common.csv
'RS_MF_IALS 41,8%.ipynb'   submission_Round_Robin_3SLIM1MF_Common.csv
 RS_PureSVD.ipynb          submission_Round_Robin.csv
 RS_SLIM_47%.ipynb         submission_SLIM_full_URM.csv


In [8]:
#Loading dataframes
URM_dataframe = pd.read_csv('RSDs/data_train.csv')
ICM_genre_dataframe=pd.read_csv('RSDs/data_ICM_genre.csv')
ICM_subgenre_dataframe=pd.read_csv('RSDs/data_ICM_subgenre.csv')
ICM_event_dataframe=pd.read_csv('RSDs/data_ICM_event.csv')
ICM_channel_dataframe=pd.read_csv('RSDs/data_ICM_channel.csv')

In [9]:
#Column Renaming
URM_dataframe.columns = ['User','Item','Interaction']
ICM_genre_dataframe.columns = ['Item','Feature','Val']
ICM_subgenre_dataframe.columns = ['Item','Feature','Val']
ICM_event_dataframe.columns = ['Item','Feature','Val']
ICM_channel_dataframe.columns = ['Item','Feature','Val']

In [10]:
#Converting datatype
URM_dataframe['Interaction'] = URM_dataframe['Interaction'].astype(int)
ICM_genre_dataframe['Val'] = ICM_genre_dataframe['Val'].astype(int)
ICM_subgenre_dataframe['Val'] = ICM_subgenre_dataframe['Val'].astype(int)
ICM_event_dataframe['Val'] = ICM_event_dataframe['Val'].astype(int)
ICM_channel_dataframe['Val'] = ICM_channel_dataframe['Val'].astype(int)

In [11]:
#Printing stats to see if everything went well
userID_unique = URM_dataframe["User"].unique()
itemID_unique = URM_dataframe["Item"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_dataframe)

#Printing some stats
print("Number of users: " + str(n_users))
print("Number of items: " + str(n_items))
print("Number of interactions: " + str(n_interactions))

Number of users: 13650
Number of items: 18059
Number of interactions: 5285664


In [12]:
#Checking if re-assignment of ID's is needed
print('The maxID for user {}, The maxID for item {}'.format(max(URM_dataframe.User),max(URM_dataframe.Item)))

The maxID for user 13649, The maxID for item 18058


Re-assignment not needed

In [22]:
#Converting the URM into a sparse matrix
URM_sparse = sps.coo_matrix((URM_dataframe['Interaction'].values,(URM_dataframe['User'].values,URM_dataframe['Item'].values)))
URM_csr = URM_sparse.tocsr()
URM_csr

<13650x18059 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [23]:
#Splitting data into training and validation set (Test set is the one on Kaggle)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [24]:
#Using SLIM imported from course's libraries
recommender = SLIMElasticNetRecommender(URM_sparse) # Previously I've worked with train+validation, now I use the entire URM to obtain better performances on the test set

Hyperparameters tuning (actually not used). This procedure takes a lot of time if we use SLIM. I've written the code for other methods such as MF techniques that I used fot the hybrid RS. Of course, different RS means different parameters to be tuned.

In [25]:
from skopt.space import Real, Integer, Categorical

In [26]:
hyperparameters_range_dictionary = {
    "topK": Integer(2999,3100),
    "l1_ratio": Real(0.00022,0.00023),
    "alpha": Real(0.07,0.08)
}

In [27]:
recommender_class = SLIMElasticNetRecommender
hyperparameterSearch = SearchBayesianSkopt(recommender_class, evaluator_validation=evaluator_validation)

In [28]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],    
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_validation],     
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [29]:
output_folder_path = '/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/Results/SLIM_res/'

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 5 
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [30]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, 
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, 
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 3077, 'l1_ratio': 0.00022375159155730377, 'alpha': 0.07452037602641845}
SLIMElasticNetRecommender: Processed 1499 ( 8.3%) in 5.00 min. Items per second: 4.99
SLIMElasticNetRecommender: Processed 3087 (17.1%) in 10.00 min. Items per second: 5.14
SLIMElasticNetRecommender: Processed 4624 (25.6%) in 15.01 min. Items per second: 5.13
SLIMElasticNetRecommender: Processed 6270 (34.7%) in 20.01 min. Items per second: 5.22
SLIMElasticNetRecommender: Processed 7919 (43.9%) in 25.01 min. Items per second: 5.28
SLIMElasticNetRecommender: Processed 9573 (53.0%) in 30.01 min. Items per second: 5.32
SLIMElasticNetRecommender: Processed 11232 (62.2%) in 35.01 min. Items per second: 5.35
SLIMElasticNetRecommender: Processed 12875 (71.3%) in 40.01 min. Items per second: 5.36
SLIMElasticNetRecommender: Processed 14492 (80.2%) in 45.01 min. Items per second: 5.37
SLIMElasticNetRecommender: Processe

In [ ]:
recommender.fit(tt) # 47.33%

SLIMElasticNetRecommender: Processed 1149 ( 6.4%) in 5.00 min. Items per second: 3.83
SLIMElasticNetRecommender: Processed 2352 (13.0%) in 10.00 min. Items per second: 3.92
SLIMElasticNetRecommender: Processed 3580 (19.8%) in 15.01 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 4777 (26.5%) in 20.01 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 5960 (33.0%) in 25.01 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 7132 (39.5%) in 30.01 min. Items per second: 3.96
SLIMElasticNetRecommender: Processed 8332 (46.1%) in 35.01 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 9541 (52.8%) in 40.01 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 10737 (59.5%) in 45.02 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 11934 (66.1%) in 50.02 min. Items per second: 3.98
SLIMElasticNetRecommender: Processed 13109 (72.6%) in 55.02 min. Items per second: 3.97
SLIMElasticNetRecommender: Processed 1429

In [ ]:
%cd RSChallenge/
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge
'Copia di RS_SLIM_43%.ipynb'      Results/   RSLib/    RS_SLIM_47%.ipynb
'HYBRID SLIM Item CF 33%.ipynb'   RSData/    RSLib2/   submission_full_URM.csv


In [ ]:
target_users = pd.read_csv('RSData/data_target_users_test.csv')

In [ ]:
final_dict = {}
start_time = time.time()

for user in target_users['user_id']:
  final_dict[user] = recommender.recommend(user,cutoff=10)
end_time = time.time()
print('Time needed is {}seconds'.format(end_time-start_time))

Time needed is 17.962255239486694seconds


In [ ]:
item_list = []
for user in target_users['user_id']:
  item_list.append(' '.join(str(e) for e in final_dict[user]))

In [ ]:
submission_df = pd.DataFrame({'user_id' : target_users['user_id'],'item_list' : item_list })
submission_df['item_list'] = submission_df['item_list'].astype(str).str.strip('[]')
submission_df

,user_id,item_list
0,0,3899 5433 8465 16365 16359 2243 9130 6061 1058...
1,1,3473 940 14400 16012 2118 10118 9275 9373 640 ...
2,2,6177 1551 17330 17022 2846 15119 14229 5925 16...
3,3,8194 15551 16359 15082 8371 960 15507 16444 16...
4,4,8194 10586 6177 16494 2152 5299 9130 10954 347...
...,...,...
13645,13645,6177 11491 3200 8898 11576 11303 1259 562 7762...
13646,13646,3473 7867 8635 16640 409 11504 17266 17622 176...
13647,13647,1846 7716 11227 7606 1186 8635 14868 11491 168...
13648,13648,6177 8635 13295 7295 14358 14181 6214 2482 179...


In [ ]:
%ls
%cd ..

'Copia di RS_SLIM_43%.ipynb'      Results/   RSLib/    RS_SLIM_47%.ipynb
'HYBRID SLIM Item CF 33%.ipynb'   RSData/    RSLib2/   submission_full_URM.csv
/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
submission_df.to_csv('RSChallenge/submission_SLIM_tuned.csv', index=False)